In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [61]:
rounds = pd.read_csv("rounds2.csv", encoding = "ISO-8859-1")
companies=pd.read_csv("companies2.csv", encoding = "ISO-8859-1")
mapping = pd.read_csv("mapping.csv", encoding = "ISO-8859-1")

In [62]:
#printing first five values of rounds dataset
rounds.head()

,ï»¿company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [63]:
rounds.dtypes


ï»¿company_permalink        object
funding_round_permalink     object
funding_round_type          object
funding_round_code          object
funded_at                   object
raised_amount_usd          float64
dtype: object

In [64]:
rounds.rename(columns = {'ï»¿company_permalink':'Company_Link'}, inplace = True)


In [65]:
rounds['Company_Link'] = rounds['Company_Link'].str.lower()


In [66]:
rounds['Company_Link'].nunique()


66373

In [67]:
companies.head()


,ï»¿permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010


In [68]:
companies.rename(columns = {'ï»¿permalink':'Company_Link'}, inplace = True)


In [69]:
companies['Company_Link'] = companies['Company_Link'].str.lower()


In [70]:
companies['Company_Link'].nunique()


66368

In [71]:
companies['Company_Link'].count()


66368

In [72]:
companies.Company_Link.isin(rounds.Company_Link).astype(int)


0        1
1        1
2        1
3        1
4        1
        ..
66363    1
66364    1
66365    0
66366    1
66367    0
Name: Company_Link, Length: 66368, dtype: int32

In [73]:
companies['category']=companies['category_list'].str.split('|').str[0]


In [74]:
companies.head()


,Company_Link,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,category
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,Media
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,Application Platforms
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,Apps
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,Curated Web
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010,Software


In [75]:
companies['category_list'] = companies['category_list'].str.lower()


In [76]:
eng_speaking = companies.country_code.loc[companies['category_list'].str.contains("english-speaking",na=False, case=True)].value_counts()


In [77]:
print(eng_speaking)


USA    2
GBR    2
JPN    2
CHN    2
SGP    1
FRA    1
AUS    1
ARG    1
ISR    1
DEU    1
Name: country_code, dtype: int64


In [78]:
companies.drop('category_list', inplace=True, axis=1)


In [79]:
master_frame = pd.merge(rounds,companies,on='Company_Link')


In [80]:
master_frame.dtypes


Company_Link                object
funding_round_permalink     object
funding_round_type          object
funding_round_code          object
funded_at                   object
raised_amount_usd          float64
name                        object
homepage_url                object
status                      object
country_code                object
state_code                  object
region                      object
city                        object
founded_at                  object
category                    object
dtype: object

In [81]:
master_frame2 = pd.merge(rounds,companies,on='Company_Link', how='right')


In [82]:
master_frame2.shape


(114943, 15)

In [83]:
(rounds.groupby(['funding_round_type'])['raised_amount_usd'].sum())


funding_round_type
angel                    4.659255e+09
convertible_note         2.103126e+09
debt_financing           1.138848e+11
equity_crowdfunding      6.611162e+08
grant                    9.134424e+09
non_equity_assistance    3.001782e+07
post_ipo_debt            2.564309e+10
post_ipo_equity          5.161061e+10
private_equity           1.419254e+11
product_crowdfunding     5.261686e+08
secondary_market         2.389489e+09
seed                     1.698986e+10
undisclosed              3.044143e+10
venture                  5.901262e+11
Name: raised_amount_usd, dtype: float64

In [84]:
(rounds.groupby(['funding_round_type'])['raised_amount_usd'].sum())/(rounds.groupby(['funding_round_type'])['raised_amount_usd'].count())


funding_round_type
angel                    9.586945e+05
convertible_note         1.453439e+06
debt_financing           1.704353e+07
equity_crowdfunding      5.383682e+05
grant                    4.300576e+06
non_equity_assistance    4.112031e+05
post_ipo_debt            1.687046e+08
post_ipo_equity          8.218249e+07
private_equity           7.330859e+07
product_crowdfunding     1.363131e+06
secondary_market         7.964963e+07
seed                     7.198180e+05
undisclosed              1.924237e+07
venture                  1.174895e+07
Name: raised_amount_usd, dtype: float64

In [85]:
master_frame.head()


,Company_Link,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,name,homepage_url,status,country_code,state_code,region,city,founded_at,category
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,#fame,http://livfame.com,operating,IND,16,Mumbai,Mumbai,NaN,Media
1,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,:Qounter,http://www.qounter.com,operating,USA,DE,DE - Other,Delaware City,04-09-2014,Application Platforms
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0,:Qounter,http://www.qounter.com,operating,USA,DE,DE - Other,Delaware City,04-09-2014,Application Platforms
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,operating,NaN,NaN,NaN,NaN,NaN,Apps
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,0-6.com,http://www.0-6.com,operating,CHN,22,Beijing,Beijing,01-01-2007,Curated Web


In [86]:
mapping.dtypes

category_list                              object
Automotive & Sports                         int64
Blanks                                      int64
Cleantech / Semiconductors                  int64
Entertainment                               int64
Health                                      int64
Manufacturing                               int64
News, Search and Messaging                  int64
Others                                      int64
Social, Finance, Analytics, Advertising     int64
dtype: object

In [87]:
category = pd.get_dummies(mapping).idxmax(1)


In [88]:
mapping.drop(mapping.columns.difference(['category_list']), 1, inplace=True)


C:\Users\kalpit\AppData\Local\Temp\ipykernel_892\1252479177.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  mapping.drop(mapping.columns.difference(['category_list']), 1, inplace=True)


In [89]:
mapping['category'] = category


In [90]:
mapping.head()


,category_list,category
0,NaN,Blanks
1,3D,Manufacturing
2,3D Printing,Manufacturing
3,3D Technology,Manufacturing
4,Accounting,"Social, Finance, Analytics, Advertising"


In [91]:
master_frame3 = pd.merge(master_frame,mapping,on='category')


In [104]:
master_frame3.shape

(44927, 16)

In [93]:
master_frame.dtypes

Company_Link                object
funding_round_permalink     object
funding_round_type          object
funding_round_code          object
funded_at                   object
raised_amount_usd          float64
name                        object
homepage_url                object
status                      object
country_code                object
state_code                  object
region                      object
city                        object
founded_at                  object
category                    object
dtype: object

In [94]:
mapping.shape

(688, 2)

In [95]:
mapping.dtypes

category_list    object
category         object
dtype: object

In [96]:
master_frame3.head()

,Company_Link,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,name,homepage_url,status,country_code,state_code,region,city,founded_at,category,category_list
0,/organization/1-4-all,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,21-04-2013,NaN,1-4 All,NaN,operating,USA,NC,NC - Other,Connellys Springs,NaN,Entertainment,Art
1,/organization/1-4-all,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,21-04-2013,NaN,1-4 All,NaN,operating,USA,NC,NC - Other,Connellys Springs,NaN,Entertainment,Artists Globally
2,/organization/1-4-all,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,21-04-2013,NaN,1-4 All,NaN,operating,USA,NC,NC - Other,Connellys Springs,NaN,Entertainment,Audio
3,/organization/1-4-all,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,21-04-2013,NaN,1-4 All,NaN,operating,USA,NC,NC - Other,Connellys Springs,NaN,Entertainment,Audiobooks
4,/organization/1-4-all,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,21-04-2013,NaN,1-4 All,NaN,operating,USA,NC,NC - Other,Connellys Springs,NaN,Entertainment,Brewing


In [97]:
master_frame3.head()


,Company_Link,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,name,homepage_url,status,country_code,state_code,region,city,founded_at,category,category_list
0,/organization/1-4-all,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,21-04-2013,NaN,1-4 All,NaN,operating,USA,NC,NC - Other,Connellys Springs,NaN,Entertainment,Art
1,/organization/1-4-all,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,21-04-2013,NaN,1-4 All,NaN,operating,USA,NC,NC - Other,Connellys Springs,NaN,Entertainment,Artists Globally
2,/organization/1-4-all,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,21-04-2013,NaN,1-4 All,NaN,operating,USA,NC,NC - Other,Connellys Springs,NaN,Entertainment,Audio
3,/organization/1-4-all,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,21-04-2013,NaN,1-4 All,NaN,operating,USA,NC,NC - Other,Connellys Springs,NaN,Entertainment,Audiobooks
4,/organization/1-4-all,/funding-round/e97a192e13ea0ee3c4f71136b4f3ec16,equity_crowdfunding,NaN,21-04-2013,NaN,1-4 All,NaN,operating,USA,NC,NC - Other,Connellys Springs,NaN,Entertainment,Brewing


In [98]:
master_frame3.shape


(121105, 16)

In [99]:
master_frame3.dropna(how='all')     #drop row when all the row elements is null
master_frame3 = master_frame3.dropna(subset=['raised_amount_usd'])       #drop row whose raised_amount_usd is 0
master_frame3.dropna(how='all', axis=1) 

,Company_Link,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,name,homepage_url,status,country_code,state_code,region,city,founded_at,category,category_list
89,/organization/1-of-99,/funding-round/b5c811f561cfb5701cd5b1762120bbc3,convertible_note,NaN,13-12-2014,100000.0,1 of 99,NaN,operating,USA,CA,SF Bay Area,Mountain View,01-12-2014,Entertainment,Art
90,/organization/1-of-99,/funding-round/b5c811f561cfb5701cd5b1762120bbc3,convertible_note,NaN,13-12-2014,100000.0,1 of 99,NaN,operating,USA,CA,SF Bay Area,Mountain View,01-12-2014,Entertainment,Artists Globally
91,/organization/1-of-99,/funding-round/b5c811f561cfb5701cd5b1762120bbc3,convertible_note,NaN,13-12-2014,100000.0,1 of 99,NaN,operating,USA,CA,SF Bay Area,Mountain View,01-12-2014,Entertainment,Audio
92,/organization/1-of-99,/funding-round/b5c811f561cfb5701cd5b1762120bbc3,convertible_note,NaN,13-12-2014,100000.0,1 of 99,NaN,operating,USA,CA,SF Bay Area,Mountain View,01-12-2014,Entertainment,Audiobooks
93,/organization/1-of-99,/funding-round/b5c811f561cfb5701cd5b1762120bbc3,convertible_note,NaN,13-12-2014,100000.0,1 of 99,NaN,operating,USA,CA,SF Bay Area,Mountain View,01-12-2014,Entertainment,Brewing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121100,/organization/zygo-corporation,/funding-round/d2c6bee0114056f3018667a9b5ab13ba,venture,NaN,24-10-2011,9000000.0,Zygo Corporation,http://www.zygo.com,acquired,USA,CT,CT - Other,Middlefield,NaN,Manufacturing,Mobile Software Tools
121101,/organization/zygo-corporation,/funding-round/d2c6bee0114056f3018667a9b5ab13ba,venture,NaN,24-10-2011,9000000.0,Zygo Corporation,http://www.zygo.com,acquired,USA,CT,CT - Other,Middlefield,NaN,Manufacturing,New Product Development
121102,/organization/zygo-corporation,/funding-round/d2c6bee0114056f3018667a9b5ab13ba,venture,NaN,24-10-2011,9000000.0,Zygo Corporation,http://www.zygo.com,acquired,USA,CT,CT - Other,Middlefield,NaN,Manufacturing,New Technologies
121103,/organization/zygo-corporation,/funding-round/d2c6bee0114056f3018667a9b5ab13ba,venture,NaN,24-10-2011,9000000.0,Zygo Corporation,http://www.zygo.com,acquired,USA,CT,CT - Other,Middlefield,NaN,Manufacturing,Optical Communications


In [100]:
master_frame3.shape


(88483, 16)

In [101]:
master_frame3=master_frame3.loc[master_frame3['funding_round_type'].isin(['venture', 'angel', 'private_equity','seed'])]


In [102]:
master_frame3=master_frame3.loc[master_frame3['funding_round_type'].isin(['venture'])]


In [103]:
master_frame3.shape


(44927, 16)

In [45]:
array = ['USA', 'FRA','CHN']
master_frame3 = master_frame3.loc[master_frame3['country_code'].isin(array)]

In [46]:
master_frame3.country_code.unique()


array(['USA', 'FRA', 'CHN'], dtype=object)

In [47]:
master_frame3.shape


(34294, 16)

In [48]:
master_frame3.loc[master_frame3["country_code"] == "USA"].Company_Link.count()


29799

In [49]:
master_frame3.loc[master_frame3["country_code"] == "FRA"].Company_Link.count()


1201

In [50]:
master_frame3.loc[master_frame3["country_code"] == "CHN"].Company_Link.count()


3294

In [51]:
master_frame3.loc[master_frame3["country_code"] == "USA"].raised_amount_usd.sum()


272385045129.0

In [52]:
master_frame3.loc[master_frame3["country_code"] == "FRA"].raised_amount_usd.sum()


63213223662.0

In [53]:
master_frame3.loc[master_frame3["country_code"] == "CHN"].raised_amount_usd.sum()


54568836840.0

In [54]:
master_frame3.groupby('country_code')['category'].value_counts()


country_code  category     
CHN           Manufacturing     2760
              Entertainment      534
FRA           Entertainment      801
              Manufacturing      400
USA           Manufacturing    19920
              Entertainment     9879
Name: category, dtype: int64

In [55]:
master_frame3.groupby('country_code')['raised_amount_usd'].max()


country_code
CHN    160000000.0
FRA    500000000.0
USA    286000000.0
Name: raised_amount_usd, dtype: float64